In this programming exercise, you will implement the validation set approach and K-fold cross validation, while recalling that leave-one-out cross validation is K-fold cross validation for K = N , if N is the number of training samples. Implement the study carried out in Example 6.3 from the lecture notes. Note that due to randomization, you might get other results than the ones shown in the example. 

In [128]:
## Importing

import numpy as np
import pandas as pd
import random
from sklearn.neighbors import KNeighborsRegressor

In [129]:
## loss function
def L2_loss(pred, actual):
    return (pred-actual)**2
    
    
## Error estimation by validation set approach
def err_validation_set(x, y):
    
    ## Splitting data into train and validate sets
    split_size = random.randint(1, len(x)-1)
    train_x = x[0:split_size]
    train_y = y[0:split_size]
    validate_x = x[split_size:]
    validate_y = y[split_size:]
    
    ## Reshaping into np array
    train_x = np.array(train_x)
    validate_x = np.array(validate_x)
    train_x = train_x.reshape(-1,1)
    validate_x = validate_x.reshape(-1,1)
    
    ## Using KNN regressor to predict for single k
#     k = 3
#     clf = KNeighborsRegressor(n_neighbors = k)
#     clf.fit(train_x, train_y)
#     prediction = (clf.predict(validate_x))
#     print(prediction, validate_y)
#     Err = (1/len(validate_x)) * sum(L2_loss(prediction, validate_y))

    ## Using KNN regressor to predict for range of k
    Err = []
    
    for k in range(len(train_x), 0, -1):
        clf = KNeighborsRegressor(n_neighbors = k)
        clf.fit(train_x, train_y)
        prediction = (clf.predict(validate_x))
        Err.append((1/len(validate_x)) * sum(L2_loss(prediction, validate_y)))
    
    return Err
    

In [130]:
## Generate random input sample
N = 51
x = []
y = []

for i in range(0, N):
    mu, sigma = 0, 1.15 ## mean and standard deviation
    noise = np.random.normal(mu, sigma, 1)
    
    temp_x = random.randint(-4,4)
    temp_y = temp_x**2 + noise
    
    x.append(temp_x)
    y.append(temp_y)
    
## Error Validation Set     
err_validation_set(x, y)

39
38
37
36
35
34
33
32
31
30
29
28
27
26
25
24
23
22
21
20
19
18
17
16
15
14
13
12
11
10
9
8
7
6
5
4
3
2
1


[array([34.3864989]),
 array([34.76636717]),
 array([35.35248856]),
 array([35.84121845]),
 array([36.34454431]),
 array([36.91756735]),
 array([37.66780226]),
 array([38.62719821]),
 array([39.04974215]),
 array([39.60792777]),
 array([40.39204013]),
 array([40.5443564]),
 array([39.85396009]),
 array([39.99747127]),
 array([38.59303486]),
 array([36.29453221]),
 array([34.52071252]),
 array([31.93742083]),
 array([29.67180101]),
 array([27.24932844]),
 array([25.81411164]),
 array([23.19353596]),
 array([20.84776656]),
 array([19.51958681]),
 array([16.7558402]),
 array([14.43702805]),
 array([12.68776593]),
 array([11.41004932]),
 array([10.2671297]),
 array([8.05359502]),
 array([7.15823276]),
 array([7.08568917]),
 array([6.57810928]),
 array([6.32362483]),
 array([4.99907801]),
 array([4.29745597]),
 array([3.08807471]),
 array([2.76295184]),
 array([4.51593667])]